In [ ]:
# PageRank Analysis for Neuronal Directed Graphs
# Based on Mean Edge Weights (assumed positive)

import pandas as pd
import networkx as nx
import ast

# ==== USER INPUT: Specify file paths here ====
edge_weights_path = r"/your/path/to/segment_edge_weights.xlsx"  # <-- CHANGE THIS
output_path = r"/your/path/to/pagerank_segment.xlsx"            # <-- CHANGE THIS

# ==== FUNCTION DEFINITION ====
def compute_pagerank(edge_weights_path, output_path='pagerank.xlsx'):
    """
    Compute PageRank for a directed neuronal network based on positive mean edge weights.

    This function builds a directed graph where each node is a neuron and each edge 
    represents a directed connection with a weight corresponding to the mean functional connectivity 
    (e.g., derived from dot products of firing rates). PageRank is computed to quantify the 
    influence of each neuron, under the assumption that connections from high-scoring neurons 
    contribute more to the score than those from low-scoring ones.
    
    Parameters
    ----------
    edge_weights_path : str
        Path to Excel file containing 'Neuron Pair' and 'Mean Edge Weight' columns.
        'Neuron Pair' must be string-formatted tuples, e.g., "(1, 2)".

    output_path : str, optional
        File path to save the resulting PageRank table. Default is 'pagerank.xlsx'.

    Returns
    -------
    pagerank_df : pandas.DataFrame
        DataFrame containing each neuron and its corresponding PageRank score.

    Notes
    -----
    This function assumes the input file contains data from a single animal. 
    All neuron pairs are treated as part of a single directed graph, and PageRank 
    is computed across that network. If you have data from multiple animals in one file, 
    you must preprocess the file to split by animal and call this function separately 
    for each subset.
    """
    # Load edge weight data
    edge_weights_df = pd.read_excel(edge_weights_path)

    # Convert string-formatted neuron pairs to tuples safely
    edge_weights_df['Neuron Pair'] = edge_weights_df['Neuron Pair'].apply(ast.literal_eval)

    # Build directed graph and add weighted edges
    G = nx.DiGraph()
    for _, row in edge_weights_df.iterrows():
        source, target = row['Neuron Pair']
        weight = row['Mean Edge Weight']
        G.add_edge(source, target, weight=weight)

    # Compute PageRank using weighted edges
    pagerank_scores = nx.pagerank(G, weight='weight')

    # Convert to DataFrame and save
    pagerank_df = pd.DataFrame(pagerank_scores.items(), columns=['Neuron', 'PageRank'])
    pagerank_df.to_excel(output_path, index=False)

    return pagerank_df

# ==== RUN THE FUNCTION AND PRINT OUTPUT ====
pagerank_df = compute_pagerank(edge_weights_path, output_path)
print(pagerank_df)


In [ ]:
# Weighted Clustering Coefficient per Neuron per Animal
# Measures how strongly each neuron's neighbors are interconnected using edge weights.
# Computed on an undirected graph using NetworkX.

import pandas as pd
import networkx as nx
import ast

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_all_animals.xlsx"  # <-- CHANGE THIS
output_path = r"/your/path/to/weighted_clustering_all_animals.xlsx"      # <-- CHANGE THIS

# === FUNCTION DEFINITION ===
def compute_weighted_clustering(edge_weights_path, output_path='weighted_clustering.xlsx'):
    """
    Compute the weighted clustering coefficient for each neuron across animals.

    Builds an undirected graph per animal from functional connectivity data and computes
    the weighted clustering coefficient, which quantifies how strongly a neuron's neighbors 
    are interconnected, accounting for edge weights. By calculating the weighted clustering coefficient for each neuron, 
    we can identify which neurons form tightly connected clusters.

    Parameters
    ----------
    edge_weights_path : str
        Path to Excel file containing columns:
        - 'Animal': Animal identifier
        - 'Neuron Pair': Tuple of neurons as a string (e.g., "(1, 2)")
        - 'Mean Edge Weight': Positive float weight between neurons

    output_path : str, optional
        Path to save the combined clustering results. Default is 'weighted_clustering.xlsx'.

    Returns
    -------
    final_results_df : pandas.DataFrame
        DataFrame with columns: Neuron, Weighted Clustering Coefficient, Animal

    Notes
    -----
    This function handles multiple animals within a single input sheet by grouping 
    based on the 'Animal' column. If you are processing one animal at a time, use a 
    version of this function that skips the groupby step.
    """
    # Load edge weight data
    df = pd.read_excel(edge_weights_path)

    # Safely parse neuron pairs
    df['Neuron Pair'] = df['Neuron Pair'].apply(ast.literal_eval)

    # Store results
    all_results = []

    # Group by animal and compute clustering coefficient for each
    for animal, group in df.groupby('Animal'):
        G = nx.Graph()
        for _, row in group.iterrows():
            source, target = row['Neuron Pair']
            weight = row['Mean Edge Weight']
            G.add_edge(source, target, weight=weight)

        clustering = nx.clustering(G, weight='weight')
        clustering_df = pd.DataFrame(clustering.items(), columns=['Neuron', 'Weighted Clustering Coefficient'])
        clustering_df['Animal'] = animal
        all_results.append(clustering_df)

    final_results_df = pd.concat(all_results, ignore_index=True)
    final_results_df.to_excel(output_path, index=False)

    return final_results_df

# === RUN THE FUNCTION AND PRINT OUTPUT ===
clustering_df = compute_weighted_clustering(edge_weights_path, output_path)
print(clustering_df)


In [ ]:
# Weighted Degree Centrality (Raw and Normalized)
# Measures total edge weight per neuron in an undirected graph, per animal

import pandas as pd
import networkx as nx
import ast

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_all_animals.xlsx"  # <-- CHANGE THIS
output_path = r"/your/path/to/weighted_degree_centrality_all_animals.xlsx"  # <-- CHANGE THIS

# === FUNCTION DEFINITION ===
def compute_weighted_degree_centrality(edge_weights_path, output_path='weighted_degree_centrality.xlsx'):
    """
    Compute raw and normalized weighted degree centrality for each neuron, grouped per animal.

    This function assumes the input file contains data for multiple animals, with an 'Animal' column
    that identifies which edges belong to which animal. A separate undirected graph is constructed
    for each animal based on its neuron pairs and edge weights. Weighted degree centrality is computed 
    as the sum of edge weights per neuron. The normalized value is calculated by dividing each neuron's 
    weighted degree by the total number of neurons in that animal's network.

    Parameters
    ----------
    edge_weights_path : str
        Path to Excel file containing:
        - 'Animal': Animal identifier (e.g., "WT1", "ELS2")
        - 'Neuron Pair': Tuple as string (e.g., "(1, 2)")
        - 'Mean Edge Weight': Positive float

    output_path : str, optional
        File path to save output Excel file. Default is 'weighted_degree_centrality.xlsx'.

    Returns
    -------
    final_results_df : pandas.DataFrame
        DataFrame with columns:
        - 'Neuron'
        - 'Weighted Degree'
        - 'Normalized Weighted Degree'
        - 'Animal'

    Notes
    -----
    This function handles multiple animals within a single input sheet by grouping 
    based on the 'Animal' column. If you are processing one animal at a time, use a 
    version of this function that skips the groupby step.
    """

    df = pd.read_excel(edge_weights_path)
    df['Neuron Pair'] = df['Neuron Pair'].apply(ast.literal_eval)

    all_results = []

    for animal, group in df.groupby('Animal'):
        G = nx.Graph()
        for _, row in group.iterrows():
            source, target = row['Neuron Pair']
            weight = row['Mean Edge Weight']
            G.add_edge(source, target, weight=weight)

        # Compute raw weighted degree (sum of edge weights per node)
        weighted_degrees = {
            node: sum(attr['weight'] for _, attr in G[node].items())
            for node in G.nodes()
        }

        num_neurons = len(G.nodes())
        normalized_degrees = {
            node: degree / num_neurons
            for node, degree in weighted_degrees.items()
        }

        degrees_df = pd.DataFrame({
            'Neuron': list(weighted_degrees.keys()),
            'Weighted Degree': list(weighted_degrees.values()),
            'Normalized Weighted Degree': list(normalized_degrees.values())
        })
        degrees_df['Animal'] = animal
        all_results.append(degrees_df)

    final_results_df = pd.concat(all_results, ignore_index=True)
    final_results_df.to_excel(output_path, index=False)

    return final_results_df

# === RUN FUNCTION ===
degree_df = compute_weighted_degree_centrality(edge_weights_path, output_path)
print(degree_df)


In [ ]:
# HITS Algorithm for Directed Graphs in a Neural Network
# Identifies hub and authority neurons based on connectivity structure

import pandas as pd
import networkx as nx
import ast

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_single_animal.xlsx"   # <-- CHANGE THIS
hub_output_path = r"/your/path/to/hub_scores.xlsx"                          # <-- CHANGE THIS
auth_output_path = r"/your/path/to/authority_scores.xlsx"                   # <-- CHANGE THIS

# === FUNCTION DEFINITION ===
def compute_hits_scores(edge_weights_path, hub_output_path='hub_scores.xlsx', auth_output_path='authority_scores.xlsx'):
    """
    Compute HITS (Hyperlink-Induced Topic Search) scores for neurons in a directed graph.

    HITS identifies two types of central nodes:
    - Hubs: Neurons that connect to many high-authority neurons
    - Authorities: Neurons that are linked to by many good hubs

    The algorithm assumes a directed graph of positive-weighted connections between neurons.
    Note: The standard NetworkX implementation of HITS does not use edge weights — only the
    directionality of connections is considered.

    Parameters
    ----------
    edge_weights_path : str
        Path to Excel file containing:
        - 'Neuron Pair': stringified tuple (e.g., "(1, 2)")
        - 'Mean Edge Weight': positive float (used only to define edges)

    hub_output_path : str, optional
        Path to save hub scores. Default is 'hub_scores.xlsx'.

    auth_output_path : str, optional
        Path to save authority scores. Default is 'authority_scores.xlsx'.

    Returns
    -------
    hubs_df : pandas.DataFrame
        DataFrame with neurons and their Hub scores.

    authorities_df : pandas.DataFrame
        DataFrame with neurons and their Authority scores.

    Notes
    -----
    This function assumes the input file contains data from a single animal. 
    All neuron pairs are treated as part of a single directed graph. If you have multiple
    animals in one file, you must split the data and call this function separately per animal.

    Edge weights are used to define edge presence but are not considered in the HITS calculation. 
    NetworkX’s HITS implementation operates on the binary structure of the directed graph.

    Although the code does not show iterations explicitly, HITS is an iterative algorithm. 
    NetworkX handles this process internally using the following approach:

    - Initializes all hub and authority scores to 1
    - Repeatedly updates scores until convergence:
        * Each neuron's authority score becomes the sum of the hub scores of neurons pointing to it
        * Each neuron's hub score becomes the sum of the authority scores of neurons it points to
    - Scores are normalized after each iteration
    - The algorithm stops when the change in scores falls below a tolerance threshold or 
      after a maximum number of iterations (default: max_iter=100, tol=1e-8)

    To manually control convergence behavior, you can pass parameters such as `max_iter` and `tol` 
    to `nx.hits()`.
    """
    df = pd.read_excel(edge_weights_path)
    df['Neuron Pair'] = df['Neuron Pair'].apply(ast.literal_eval)

    G = nx.DiGraph()
    for _, row in df.iterrows():
        source, target = row['Neuron Pair']
        G.add_edge(source, target)  # weights ignored by nx.hits()

    hubs, authorities = nx.hits(G)

    hubs_df = pd.DataFrame(hubs.items(), columns=['Neuron', 'Hub Score'])
    authorities_df = pd.DataFrame(authorities.items(), columns=['Neuron', 'Authority Score'])

    hubs_df.to_excel(hub_output_path, index=False)
    authorities_df.to_excel(auth_output_path, index=False)

    return hubs_df, authorities_df

# === RUN FUNCTION ===
hubs_df, authorities_df = compute_hits_scores(edge_weights_path, hub_output_path, auth_output_path)

print("Hub Scores:\n", hubs_df)
print("\nAuthority Scores:\n", authorities_df)


In [ ]:
# Leiden Community Detection for Undirected Neural Graphs
# Identifies clusters of tightly interconnected neurons using modularity optimization

import pandas as pd
import igraph as ig
import leidenalg as la
import numpy as np
import ast
from IPython.display import Image, display

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_single_animal.xlsx"  # <-- CHANGE THIS
output_image_path = "community_detection.png"  # <-- CHANGE IF NEEDED

# === FUNCTION DEFINITION ===
def detect_communities_leiden(edge_weights_path, image_path="community_detection.png"):
    """
    Detects neuronal communities using the Leiden algorithm on an undirected graph.

    The graph is constructed from mean edge weights between neuron pairs. 
    The Leiden algorithm partitions the graph to maximize modularity, identifying
    clusters of tightly interconnected neurons.

    Parameters
    ----------
    edge_weights_path : str
        Path to Excel file containing:
        - 'Neuron Pair': string-formatted tuple (e.g., "(1, 2)")
        - 'Mean Edge Weight': positive float
    
    image_path : str, optional
        Path to save the community detection visualization. Default is 'community_detection.png'.

    Returns
    -------
    partition : leidenalg.VertexPartition
        The community partition object.

    Notes
    -----
    This function assumes the input file contains data from a single animal.
    If you have multiple animals in one file, you must preprocess and run separately.

    The graph is undirected. Edge weights are used for modularity optimization,
    but directionality is ignored.

    The Leiden algorithm is chosen over Louvain and other community detection methods
    due to its superior performance in detecting well-separated and stable communities.
    Compared to Louvain, Leiden:
    - Ensures communities are locally well-connected (no disconnected parts)
    - Provides faster convergence and better modularity optimization
    - Is more robust to resolution parameter changes
    These advantages are particularly important for dynamic neural data where 
    communities may shift across time and subtle changes in modular structure matter.
    """
    # Load data and safely parse tuples
    df = pd.read_excel(edge_weights_path)
    df['Neuron Pair'] = df['Neuron Pair'].apply(ast.literal_eval)

    # Extract source, target, and weight info
    sources = df['Neuron Pair'].apply(lambda x: x[0]).tolist()
    targets = df['Neuron Pair'].apply(lambda x: x[1]).tolist()
    weights = df['Mean Edge Weight'].tolist()

    # Build undirected igraph
    g = ig.Graph.TupleList(edges=list(zip(sources, targets)), directed=False)
    g.es['weight'] = weights

    # Apply Leiden algorithm
    partition = la.find_partition(g, la.ModularityVertexPartition, weights='weight')

    # Visualization
    def plot_graph(graph, partition, weights, image_path):
        percentiles = np.percentile(weights, np.arange(0, 100, 0.1))
        edge_widths = []
        edge_colors = []
        min_w, max_w = min(weights), max(weights)

        for weight in weights:
            norm_weight = (weight - min_w) / (max_w - min_w + 1e-10)
            edge_widths.append(norm_weight * 20 + 0.3)
            if weight > 0:
                percentile_rank = next((x[0] for x in enumerate(percentiles) if x[1] >= weight), len(percentiles) - 1) / 1000.0
                intensity = percentile_rank * 0.9 + 0.1
                edge_colors.append((0, 0, 0, intensity))
            else:
                edge_colors.append((0, 0, 0, 0))

        node_colors = ['red' if sum(graph.es.select(_source=v.index)['weight']) == 0 else 'green' for v in graph.vs]

        visual_style = {
            "vertex_size": 40,
            "vertex_color": node_colors,
            "edge_color": edge_colors,
            "edge_width": edge_widths,
            "vertex_label": list(range(1, graph.vcount() + 1)),
            "bbox": (800, 800),
            "margin": 100,
        }

        ig.plot(graph, image_path, **visual_style)
        return Image(filename=image_path_


In [ ]:
# Eigenvector Centrality per Neuron per Animal
# Measures influence of each neuron based on connections to other high-centrality neurons

import pandas as pd
import networkx as nx
import ast

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_all_animals.csv"  # <-- CHANGE THIS
output_path = r"/your/path/to/eigenvector_centrality_results.csv"        # <-- CHANGE THIS

# === FUNCTION DEFINITION ===
def compute_eigenvector_centrality(edge_weights_path, output_path="eigenvector_centrality_results.csv"):
    """
    Compute eigenvector centrality for each neuron in an undirected, weighted graph per animal.

    Eigenvector centrality measures a neuron's influence based on its connections to other 
    high-scoring neurons. It is computed per animal using the 'Animal' column.

    Parameters
    ----------
    edge_weights_path : str
        Path to CSV file containing:
        - 'Animal': Animal identifier (e.g., "WT1", "ELS2")
        - 'Neuron Pair': stringified tuple (e.g., "(1, 3)")
        - 'Mean Edge Weight': positive float

    output_path : str, optional
        Path to save the resulting centrality scores as a CSV file.

    Returns
    -------
    eigenvector_df : pandas.DataFrame
        DataFrame with columns:
        - 'Animal'
        - 'Neuron'
        - 'Eigenvector Centrality'

    Notes
    -----
    This function assumes the input file includes multiple animals. 
    Graphs are built per animal using undirected edges with weights.
    If convergence fails (e.g., due to disconnected components), the result for that animal is skipped.
    """
    data = pd.read_csv(edge_weights_path)
    data['Mean Edge Weight'] = pd.to_numeric(data['Mean Edge Weight'], errors='coerce')
    data['Neuron Pair'] = data['Neuron Pair'].apply(ast.literal_eval)

    results = []

    for animal, group in data.groupby('Animal'):
        G = nx.Graph()
        for _, row in group.iterrows():
            source, target = row['Neuron Pair']
            weight = row['Mean Edge Weight']
            G.add_edge(source, target, weight=weight)

        if G.number_of_edges() == 0:
            print(f"Warning: No edges found for Animal {animal}.")
            continue

        try:
            eigen_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000, tol=1e-06)
        except nx.PowerIterationFailedConvergence:
            print(f"Warning: Eigenvector Centrality failed to converge for Animal {animal}.")
            continue

        for node, score in eigen_centrality.items():
            results.append({
                'Animal': animal,
                'Neuron': node,
                'Eigenvector Centrality': score
            })

    eigenvector_df = pd.DataFrame(results)
    eigenvector_df.to_csv(output_path, index=False)
    return eigenvector_df

# === RUN FUNCTION ===
eigenvector_df = compute_eigenvector_centrality(edge_weights_path, output_path)

print("Eigenvector Centrality Results:")
print(eigenvector_df.head())


In [ ]:
# Weighted Global and Local Efficiency per Animal
# Uses distance = 1 - weight for functional connectivity interpretation

import pandas as pd
import networkx as nx
import numpy as np
import ast

# === USER INPUT ===
edge_weights_path = r"/your/path/to/mean_edge_weights_all_animals.csv"  # <-- CHANGE THIS
output_path = r"/your/path/to/baseline_efficiency_results_animal_level.csv"  # <-- CHANGE THIS

# === EFFICIENCY FUNCTIONS ===
def global_efficiency(graph):
    n = len(graph)
    if n < 2:
        return 0
    efficiency_sum = 0
    for _, target_lengths in nx.all_pairs_dijkstra_path_length(graph, weight='weight'):
        for dist in target_lengths.values():
            if dist > 0:
                efficiency_sum += 1 / dist
    return efficiency_sum / (n * (n - 1))

def local_efficiency(graph):
    efficiencies = []
    for v in graph.nodes:
        neighbors = list(graph.neighbors(v))
        if len(neighbors) > 1:
            subgraph = graph.subgraph(neighbors)
            efficiencies.append(global_efficiency(subgraph))
    return np.mean(efficiencies) if efficiencies else 0

# === MAIN FUNCTION ===
def compute_efficiency_metrics(edge_weights_path, output_path="efficiency_results.csv"):
    """
    Compute global and local efficiency for each animal using functional connectivity-based distances.

    Global efficiency measures how efficiently information is exchanged across the network.
    Local efficiency measures how efficiently information is exchanged within local neighborhoods.
    This implementation uses a biologically grounded transformation of functional connectivity 
    (bounded between 0 and 1) into distance values.

    Parameters
    ----------
    edge_weights_path : str
        Path to CSV file with:
        - 'Animal': Animal identifier
        - 'Neuron Pair': string-formatted tuple, e.g., "(1, 2)"
        - 'Mean Edge Weight': normalized functional connectivity between neurons (range: 0 to 1)

    output_path : str, optional
        Path to save the resulting efficiency metrics. Default is 'efficiency_results.csv'.

    Returns
    -------
    results_df : pandas.DataFrame
        DataFrame with columns: 'Animal', 'Global Efficiency', 'Local Efficiency'

    Notes
    -----
    - Graphs are undirected and constructed per animal
    - Edge weights represent functional connectivity (0 to 1), with higher values 
      indicating stronger connections
    - To convert connectivity to distance for efficiency calculations, we use:
          distance = 1 - weight
      so that stronger connections result in shorter distances and higher efficiency
    - If the graph is disconnected, only the largest connected component is used
    """
    data = pd.read_csv(edge_weights_path)
    data['Mean Edge Weight'] = pd.to_numeric(data['Mean Edge Weight'], errors='coerce')
    data['Neuron Pair'] = data['Neuron Pair'].apply(ast.literal_eval)

    results = []

    for animal, group in data.groupby('Animal'):
        G = nx.Graph()
        for _, row in group.iterrows():
            source, target = row['Neuron Pair']
            weight = row['Mean Edge Weight']
            if 0 <= weight <= 1:
                distance = 1 - weight
                G.add_edge(source, target, weight=distance)

        if G.number_of_edges() == 0:
            global_eff = 0
            local_eff = 0
        else:
            if not nx.is_connected(G):
                largest_component = max(nx.connected_components(G), key=len)
                G = G.subgraph(largest_component).copy()

            global_eff = global_efficiency(G)
            local_eff = local_efficiency(G)

        results.append({
            'Animal': animal,
            'Global Efficiency': global_eff,
            'Local Efficiency': local_eff
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)
    return results_df

# === RUN FUNCTION ===
results_df = compute_efficiency_metrics(edge_weights_path, output_path)

print("Efficiency Metrics:")
print(results_df)
